# Projecte Final - Python

In [2]:
import TwitterSentimentAnalysis as tsa
import TwitterSentimentAnalysis.data_processing as dp
import nltk
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
df_train, df_test = tsa.data_processing.load_data()

## Arreglando el texto

Revisando el texto, podemos ver que hay varias anomalías en el texto que se deben arreglar:

1. Hay 'whitespace' extra en algunos de los tweets.
2. Existens secuencias / carácteres anómalos como "&lt----" y "&quot;" en algunos tweets.

In [3]:
df_train = df_train.replace('&quot;', "'", regex=True)
df_train = df_train.replace('&lt-+', "", regex=True)

df_test = df_test.replace('&quot;', "'", regex=True)
df_test = df_test.replace('&lt-+', "", regex=True)

In [4]:
df_train['SentimentText'] = df_train['SentimentText'].map(lambda x: x.strip())
df_test['SentimentText'] = df_test['SentimentText'].map(lambda x: x.strip())

## Extrayendo características

Se debería revisar el tokenizador utilizado, algunas de las features conservan signos de puntuación añadidos.

Además sería ideal separar las características en sets positivos y negativos.

In [9]:
all_words = dp.get_tweets_words(df_train)

NameError: name 'df_train' is not defined

In [8]:
# Filtrado de palabras sin significado como artículos, determinantes...
stop_words = set(stopwords.words('english'))
features = dp.get_word_features(all_words)
filtered_features = [w for w in all_words if not w in stop_words]

NameError: name 'all_words' is not defined

In [40]:
print(len(all_words))
print(len(filtered_features))

19357
14261


In [42]:
df_test.head()

,ItemID,Sentiment,SentimentSource,SentimentText
1301,1302,1,Sentiment140,"I AM BORED, and ew. My dog just killed an insect!"
1302,1303,0,Sentiment140,i am crying you all a river...
1303,1304,1,Sentiment140,I am falling in love with Stockington - such a...
1304,1305,1,Sentiment140,i am musical today!
1305,1306,0,Sentiment140,I am not awake yet and I don't get to workout ...


Viendo las "features" que nos quedan, aquello que se debe hacer es:

1. 'Lemmatizing' de las features
2. Filtrar NEs (Named Entities)

In [6]:
def extract_features(doc):
    doc_words = set(doc)
    features = {}
    for word in filtered_features:
        features['contains(%s)' % word] = (word in doc_words), 
    return features

In [10]:
train_tweets = dp.get_tweet_and_sentiment(df_test)

In [11]:
training_set = nltk.classify.apply_features(extract_features, train_tweets)

## Entrenando un clasificador bayesiano ingenuo

El clasificador bayesiano ingenuo (Naive Bayes) emplea el modelo "bag-of-words". Esto quiere decir que contará todas las palabras del documento a clasificar, y las comparará con las características ya etiquetadas. Cuantas más palabras del documento aparezcan en las características positivas, más probabilidad habrá de que sea clasificado como positivo.

In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [3]:
#Restaura el clasificador guardado
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [10]:
print(classifier.show_most_informative_features(100))

Most Informative Features
          contains(late) = (True,)             1 : 0      =      8.8 : 1.0
     contains(beautiful) = (True,)             1 : 0      =      8.8 : 1.0
       contains(excited) = (True,)             1 : 0      =      8.8 : 1.0
       contains(Getting) = (True,)             1 : 0      =      8.8 : 1.0
          contains(Good) = (True,)             1 : 0      =      8.8 : 1.0
          contains(miss) = (True,)             0 : 1      =      8.3 : 1.0
          contains(name) = (True,)             1 : 0      =      6.8 : 1.0
       contains(Twitter) = (True,)             1 : 0      =      6.8 : 1.0
           contains(sun) = (True,)             1 : 0      =      6.8 : 1.0
       contains(forward) = (True,)             1 : 0      =      6.8 : 1.0
         contains(dance) = (True,)             1 : 0      =      6.8 : 1.0
           contains(sad) = (True,)             0 : 1      =      6.7 : 1.0
         contains(party) = (True,)             1 : 0      =      6.4 : 1.0

In [15]:
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

## Probando el clasificador nuevo

TODO: Realizar métricas de clasificación.

In [7]:
tweet = "Mexico know work emotional" 
print(classifier.classify(extract_features(tweet.split())))
print(extract_features(tweet.split()))

NameError: name 'filtered_features' is not defined